In [1]:
import pandas as pd

with pd.HDFStore("./recommender_data/experiment-1/knn_meta_table.h5") as store:
            meta_features_table_knn = store.get("meta_features_table")
            not_agg_evaluation_table_knn = store.get("not_agg_evaluation_table")
            evaluation_table_knn = store.get("evaluation_table")
            scaler_method_knn = store["scaler_method"].iloc[0]


with pd.HDFStore("./recommender_data/experiment-1/regression_meta_table.h5") as store:
            meta_features_table_reg = store.get("meta_features_table")
            not_agg_evaluation_table_reg = store.get("not_agg_evaluation_table")
            evaluation_table_reg = store.get("evaluation_table")
            scaler_method_reg = store["scaler_method"].iloc[0]

In [ ]:
meta_features_table_reg.to_csv("./plot_data/experiment-1/meta_features_table.csv", index=False)

AttributeError: 'DataFrame' object has no attribute 'save'

In [3]:
meta_features_table_reg

,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,can_cor.mean,can_cor.sd,cat_to_num,...,tree_imbalance.sd,tree_shape.mean,tree_shape.sd,var.mean,var.sd,var_importance.mean,var_importance.sd,w_lambda,worst_node.mean,worst_node.sd
1043_ada_agnostic,0.008076,0.068451,0.002277,0.487575,0.036792,0.561303,0.002908,0.0,0.0,0.0,...,0.358205,0.012390,0.179874,0.0,0.0,0.040575,0.082875,0.0,0.595559,0.003122
1460_banana,0.018818,0.000000,0.655553,0.000214,0.001305,0.268795,0.089752,0.0,0.0,0.0,...,0.291647,0.019560,0.150003,0.0,0.0,1.000000,0.425873,0.0,0.275539,0.097913
1462_banknote-authentication,0.079141,0.147877,0.536879,0.000009,0.010380,0.739677,0.151297,0.0,0.0,0.0,...,0.695297,0.302634,0.606986,0.0,0.0,0.498967,0.562337,0.0,0.215528,0.006171
1466_cardiotocography,0.041816,0.188793,0.285023,0.825212,0.058818,1.000000,0.000000,0.0,0.0,0.0,...,0.000000,1.000000,0.000000,0.0,0.0,0.055195,0.389382,0.0,0.548738,0.006181
1475_first-order-theorem-proving,0.159952,0.552636,0.515038,0.544216,0.029761,0.315149,0.265705,0.0,0.0,0.0,...,0.312985,0.013647,0.147358,0.0,0.0,0.037231,0.020176,0.0,0.285434,0.107141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spambase,0.057643,0.433470,0.096516,0.562013,0.044251,0.596928,0.234005,0.0,0.0,0.0,...,0.373381,0.031128,0.271067,0.0,0.0,0.033094,0.097880,0.0,0.360850,0.187647
sylva_prior,0.008222,0.111015,0.070431,0.970933,0.026781,0.897692,0.020090,0.0,0.0,0.0,...,0.465097,0.089106,0.426712,0.0,0.0,0.016491,0.103040,0.0,0.956467,0.001506
visualizing_soil,0.050778,0.334608,0.533369,0.996965,0.001611,0.714450,0.752608,0.0,0.0,0.0,...,1.000000,0.899746,0.383326,0.0,0.0,0.498967,0.722514,0.0,0.109386,0.514661
wind,0.043376,0.143995,0.659467,0.091685,0.007570,0.674358,0.138816,0.0,0.0,0.0,...,0.462460,0.021192,0.095032,0.0,0.0,0.141086,0.258016,0.0,0.541247,0.330399
